In [1]:
import torch as th
from ActiveCritic.utils.dataset import DatasetAC

/home/hendrik/anaconda3/envs/tfTest/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
dataset = DatasetAC(device='cuda')

In [7]:
batch_size = 5
seq_len = 5
action_dim = 3
obs_dim = 4

inpt = th.ones(size=[batch_size, seq_len, obs_dim+action_dim])
label = th.ones(size=[batch_size, seq_len, action_dim])
label[:2, -2:] = 0
reward = th.ones(size=[batch_size, seq_len], dtype=th.bool)
reward[:2, -2:] = 0

In [8]:
print(inpt.shape)

torch.Size([5, 5, 7])


In [9]:
dataset.set_data(inpt=inpt, label=label, reward=reward)

In [17]:
th.equal(dataset.success, reward[:,-1] == 1)

True

In [11]:
dataset.onyl_positiv = True
inpt, label, rew = dataset.__getitem__(index=0)

In [12]:
rew.shape

torch.Size([5])

In [13]:
dataset.reward[:,-1]

tensor([False, False,  True,  True,  True], device='cuda:0')

In [14]:
loader = th.utils.data.DataLoader(dataset=dataset, batch_size=32)

In [15]:
dataset.onyl_positiv = True

for data in loader:
    inpt, label, reward = data
    assert list(inpt.shape) == [batch_size-2, seq_len, action_dim+obs_dim]
    assert list(label.shape) == [batch_size-2, seq_len, action_dim]
    assert list(reward.shape) == [batch_size-2, seq_len]
    assert th.all(label == 1)

In [16]:
dataset.onyl_positiv = False

for data in loader:
    inpt, label, reward = data
    assert list(inpt.shape) == [batch_size, seq_len, action_dim+obs_dim]
    assert list(label.shape) == [batch_size, seq_len, action_dim]
    assert list(reward.shape) == [batch_size, seq_len]


In [18]:
dataset.add_data(inpt=inpt, label=label, reward=reward)

In [19]:
dataset.onyl_positiv = True

for data in loader:
    inpt, label, reward = data
    assert list(inpt.shape) == [2*(batch_size-2), seq_len, action_dim+obs_dim]
    assert list(label.shape) == [2*(batch_size-2), seq_len, action_dim]
    assert list(reward.shape) == [2*(batch_size-2), seq_len]
    assert th.all(label == 1)

In [20]:
dataset.onyl_positiv = False

for data in loader:
    inpt, label, reward = data
    assert list(inpt.shape) == [2*batch_size, seq_len, action_dim+obs_dim]
    assert list(label.shape) == [2*batch_size, seq_len, action_dim]
    assert list(reward.shape) == [2*batch_size, seq_len]